# KMZ from Projected Coordinate Values

Supports **PNEZD** (Point, Northing, Easting, Elevation, Description) and **PENZD** (Point, Easting, Northing, Elevation, Description).

**Outputs:**
- points.csv
- points.kmz

In [1]:
import os
import pandas as pd
from pyproj import Transformer, CRS
from zipfile import ZipFile, ZIP_DEFLATED
from pyproj import CRS

In [2]:
# ----------------------------
# Configuration
# ----------------------------
DIR = r"C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points"

# ----- Inputs (your existing variables) -----
file_name = "points.csv"       # Headerless, comma-separated
input_format = "PNEZD"                  # "PNEZD" or "PENZD"

in_crs  = "EPSG:6407"                   # e.g., NAD83(2011) / Arizona East (International foot)
out_crs = "EPSG:4326"                   # WGS84 lat/lon
altitude_mode = "absolute"              # 'absolute' | 'clampToGround' | 'relativeToGround'

# Optional: override vertical unit if different from XY (NOT COMMON, but possible)
# choices: None | 'us_survey_foot' | 'international_foot' | 'meter'
override_vertical_unit = None


In [3]:

# Directory to use (DIR should already be defined in your environment/code).
# If not defined, default to the current working directory for safety.
try:
    DIR
except NameError:
    DIR = os.getcwd()

# ----- Build input path -----
input_path = os.path.join(DIR, file_name)

# ----- Helpers -----
def sanitize_crs(crs: str) -> str:
    """
    Convert CRS identifier to a safe, lowercase token for filenames:
    - lowercases everything
    - replaces ':' with '-'
    - strips surrounding whitespace
    """
    return crs.strip().lower().replace(":", "-")

def base_name_without_ext(name: str) -> str:
    """
    Return the lowercased base name without extension.
    """
    base, _ext = os.path.splitext(name)
    return base.strip().lower()

# ----- Derive dynamic outputs -----
san_out_crs = sanitize_crs(out_crs)
base_no_ext = base_name_without_ext(file_name)

# Output CSV: file_name_{out_crs}.csv (lowercase, ':' -> '-')
output_csv = os.path.join(DIR, f"{base_no_ext}_{san_out_crs}.csv")

# Output KMZ: file_name.kmz (based on input base name, lowercase)
kmz_path = os.path.join(DIR, f"{base_no_ext}.kmz")

# ----- Show paths -----
print("Input path :", input_path)
print("Output CSV :", output_csv)
print("KMZ path   :", kmz_path)



Input path : C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points.csv
Output CSV : C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points_epsg-4326.csv
KMZ path   : C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points.kmz


In [4]:
# ----------------------------
# Helpers
# ----------------------------
def unit_factor_from_epsg(crs_code: str) -> float:
    """
    Return scale factor to convert horizontal axis units to meters.
    Uses pyproj CRS metadata instead of hardcoding EPSG mappings.
    """
    try:
        crs = CRS.from_user_input(crs_code)
        # Get the first axis (usually horizontal) unit name and conversion factor
        axis_info = crs.axis_info[0]
        unit_name = axis_info.unit_name.lower()  # e.g., 'foot', 'metre'
        print("UoM: ", unit_name)
        conversion_factor = axis_info.unit_conversion_factor  # factor to meters
        return conversion_factor
    except Exception:
        # Default to meters if CRS cannot be parsed
        return 1.0

def unit_factor_from_override(name):
    if name is None: return None
    name = name.lower()
    if name == "us_survey_foot":     return 0.3048006096012192
    if name == "international_foot": return 0.3048
    if name == "meter":              return 1.0
    raise ValueError("override_vertical_unit must be: None | 'us_survey_foot' | 'international_foot' | 'meter'")

def esc(s: str) -> str:
    return (s or "").replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

In [ ]:

# ---------- Read CSV ----------
# Expect 5 columns by position:
#   PNEZD: Point, Northing, Easting, Elevation, Description
#   PENZD: Point, Easting, Northing, Elevation, Description
df = pd.read_csv(input_path, header=None, delimiter=",")

fmt = input_format.upper()
if fmt == "PNEZD":
    df.columns = ["PID", "Northing", "Easting", "Elevation", "Description"]
elif fmt == "PENZD":
    df.columns = ["PID", "Easting", "Northing", "Elevation", "Description"]
else:
    raise ValueError("input_format must be 'PNEZD' or 'PENZD'")

# Ensure numeric (coerce bad values to NaN)
for c in ["Easting", "Northing", "Elevation"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# ---------- Vertical unit handling ----------
def unit_factor_from_override(override):
    if override is None:
        return None
    m = {
        "us_survey_foot": 1200/3937,      # 0.3048006096012192
        "international_foot": 0.3048,
        "meter": 1.0
    }
    if override not in m:
        raise ValueError("override_vertical_unit must be None | 'us_survey_foot' | 'international_foot' | 'meter'")
    return m[override]

def unit_factor_from_epsg(epsg_str):
    # Simple heuristic: most projected CRSs specify XY units; elevation assumed same unit unless override provided.
    # If you need precise vertical CRS handling, integrate explicit vertical CRS or metadata lookup.
    # For common US state plane feet:
    if "6407" in epsg_str or "ft" in epsg_str.lower():  # crude hint; adjust per your catalog
        return 1200/3937  # US survey foot
    # Default assume meters
    return 1.0

override_factor = unit_factor_from_override(override_vertical_unit)
vertical_to_meter = override_factor if override_factor is not None else unit_factor_from_epsg(in_crs)
df["Elevation_m"] = df["Elevation"] * vertical_to_meter

# ---------- Transform coordinates ----------
transformer = Transformer.from_crs(in_crs, out_crs, always_xy=True)
lon, lat, h = transformer.transform(
    df["Easting"].values, df["Northing"].values, df["Elevation_m"].values
)
df["Longitude"] = lon
df["Latitude"]  = lat
df["EllipsoidalHeight_m"] = h

# ---------- Write CSV (exclude original coordinate values) ----------
# Keep ONLY non-original coordinate identifiers + transformed outputs.
# Excluded: Easting, Northing, Elevation, Elevation_m
csv_cols = ["PID", "Longitude", "Latitude", "EllipsoidalHeight_m", "Description"]
df_out = df[csv_cols]
df_out.to_csv(output_csv, index=False)

# ---------- Build KML/KMZ ----------
def esc(s: str) -> str:
    # Minimal XML escaping
    return (
        s.replace("&", "&amp;")
         .replace("<", "&lt;")
         .replace(">", "&gt;")
    )

kml_header = (
    '<?xml version="1.0" encoding="UTF-8"?>\n'
    '<kml xmlns="http://www.opengis.net/kml/2.2">\n'
    '  <Document>\n'
    f'    <name>{base_no_ext}</name>\n'
)
kml_footer = "  </Document>\n</kml>\n"

placemarks = []
for _, row in df.iterrows():
    name = esc(str(row.get("PID", "")))           # PID as Name
    desc = esc(str(row.get("Description", "")))   # Description as Description
    lon = row["Longitude"]
    lat = row["Latitude"]
    alt = row["EllipsoidalHeight_m"] if pd.notna(row["EllipsoidalHeight_m"]) else 0
    pm = (
        "    <Placemark>\n"
        f"      <name>{name}</name>\n"
        f"      <description>{desc}</description>\n"
        "      <Point>\n"
        f"        <coordinates>{lon},{lat},{alt}</coordinates>\n"
        f"        <altitudeMode>{altitude_mode}</altitudeMode>\n"
        "      </Point>\n"
        "    </Placemark>\n"
    )
    placemarks.append(pm)

kml_text = kml_header + "".join(placemarks) + kml_footer

with ZipFile(kmz_path, "w", ZIP_DEFLATED) as z:
    z.writestr("doc.kml", kml_text.encode("utf-8"))

print(f"Wrote:\n  CSV: {output_csv}\n  KMZ: {kmz_path}")
print(df_out.head())


Wrote:
  CSV: C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points_epsg-4326.csv
  KMZ: C:\Users\USFJ139860\WSP O365\SW Geomatics Projects - US0050682.5847 - NTUA Ganado Lagoon Improvem\CAD\TOPO\XSURVEY\2019_Points\points.kmz
   PID Description   Longitude   Latitude  EllipsoidalHeight_m
0  100       PANEL -106.838415  33.371574          1938.386792
1  101       PANEL -106.837377  33.370025          1938.305715
2  102       PANEL -106.839195  33.372619          1937.802184
3  103       PANEL -106.839792  33.373632          1939.003708
4  104       PANEL -106.842988  33.368490          1928.933706
